#### Import Julia packages

In [171]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [172]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Makie
# using Interpolations

#### Import Custom functions

In [173]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

#### Read JSON File

In [174]:
width, height, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, Any[(0.75125, 2.32125, "Left"), (0.73125, 2.32125, "Right")], Any[(0.235, 0.655, "Top")], [[2.005, 3.435, 0.10125, 0.20125], [1.765, 3.215, 2.80125, 2.92125]])

In [175]:
# price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.11781, 0.11106, 0.10592, 0.10341, 0.10836, 0.10655, 0.16935, 0.43496, 0.39644], Any[1.38209, 1.3029, 1.2426, 1.21315, 1.27123, 1.24999, 1.98673, 5.10273, 4.65084], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75], Any[10, 11, 12, 13, 14, 15, 16, 17, 18])

#### Boundary conditions

In [176]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [177]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition)
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

#### Time parameters

In [178]:
t0 = 0.0    # Start time
tF = 28800   # End time
Δt = 10
# tF = 10.0
# Δt = 0.05   # Timestep

10

#### Room parameters

In [179]:
# h_wall(x) = 0.01
# h_window(x) = 0.01
# h_door(x) = 0.01

# ρ(x)=1
# c(x)=1
# k(x)=10

h_wall(x) = 0.22*0.02           # [W/m^2K]
h_window(x) = 1.2*0.02
h_door(x) = 1.0*0.02 #mindre?

ρ(x)=1.225
c(x)=1020.0
k(x)=5#3.5


h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 


Tout = get_temp_function(T_out, tF)

Tini(x)=273.15 + 22.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(Tini, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 282587541835656308

#### Controll parameter

In [180]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)

Q(x,t)=0#150*q_pos(x)*χ(t,6.0,9.0)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [181]:
# L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
# L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)

L2norm(u)=√(Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1)))
L2skp(u)=Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1))
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)


Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=400*(∑(∫(q_pos)*dΩ))
println(b)
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

100.79999999999991


Proj (generic function with 2 methods)

#### Test Gradient DEs

In [182]:
price = t -> 0

γ = 1.225*1020
# γ = 1e7
Q(x,t)= 0#χ(t,0.0,tF/2)*20.0*q_pos(x)*0.0
Qt(t)=x->Q(x,t)

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-3, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.0207905669993106e6, k = 1
α = 10, new_cost = 649655.5511486433, L2fgrad = 3910.241196448855, interm = 0.0
entered for loop, E=649655.5511486433, k = 2
α = 10, new_cost = 415926.4087286574, L2fgrad = 3114.905442979677, interm = 0.0
entered for loop, E=415926.4087286574, k = 3
α = 10, new_cost = 266308.53485314094, L2fgrad = 2495.8633779819884, interm = 0.0
entered for loop, E=266308.53485314094, k = 4
α = 10, new_cost = 170570.60685006026, L2fgrad = 1992.7830418174556, interm = 0.0
entered for loop, E=170570.60685006026, k = 5
α = 10, new_cost = 109275.84443822612, L2fgrad = 1597.8767852960314, interm = 0.0
entered for loop, E=109275.84443822612, k = 6
α = 10, new_cost = 70053.58411079677, L2fgrad = 1274.7227565028554, interm = 0.0
entered for loop, E=70053.58411079677, k = 7
α = 10, new_cost = 44935.3053289747, L2fgrad = 1023.1597391240151, interm = 0.0
entered for loop, E=44935.3053289747, k = 8
α = 10, new_cost = 28859.859335757803, L2fgrad

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (28710.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28730.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28750.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28770.0, SingleFieldFEFunction()), (28780.0, SingleFieldFEFunction()), (28790.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [183]:
price = t -> 0
c1_zero = (plot_costs2(costs))
CairoMakie.save("price_zero_convergence.png", c1_zero)


Q_int = get_control_int(qs)
T_int = get_temperature_int(Ts)
c2_zero = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_zero_result.png", c2_zero)

CairoMakie.Screen{IMAGE}


In [184]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2
γ = 1.225*1020*2

(Ts2,qs2,Ws2,costs2) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=10.0, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=2.0415811339986213e6, k = 1
α = 10.0, new_cost = 5.250070987427072e6, L2fgrad = 6971.255466816153, interm = 0.0
α = 5.0, new_cost = 1.4095916097289005e6, L2fgrad = 6971.255466816153, interm = 0.0
entered for loop, E=1.4095916097289005e6, k = 2
α = 10.0, new_cost = 2.0415811339986213e6, L2fgrad = 5584.210315584089, interm = 0.0
α = 5.0, new_cost = 934773.0338848179, L2fgrad = 5584.210315584089, interm = 0.0
entered for loop, E=934773.0338848179, k = 3
α = 10.0, new_cost = 3.439644603661912e6, L2fgrad = 4181.672545324251, interm = 0.0
α = 5.0, new_cost = 676437.5251155067, L2fgrad = 4181.672545324251, interm = 0.0
entered for loop, E=676437.5251155067, k = 4
α = 10.0, new_cost = 1.3969830152888969e6, L2fgrad = 3363.3663070908406, interm = 0.0
α = 5.0, new_cost = 471505.35198474606, L2fgrad = 3363.3663070908406, interm = 0.0
entered for loop, E=471505.35198474606, k = 5
α = 10.0, new_cost = 1.3034045548330387e6, L2fgrad = 2302.1929914424595, inter

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (28710.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28730.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28750.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28770.0, SingleFieldFEFunction()), (28780.0, SingleFieldFEFunction()), (28790.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [185]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2

# c1_step = (plot_costs(costs2))
c1_step = (plot_costs2(costs2))

CairoMakie.save("price_step_convergence.png", c1_step)


Q_int = get_control_int(qs2)
T_int = get_temperature_int(Ts2)
c2_step = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_step_result.png", c2_step)



CairoMakie.Screen{IMAGE}


In [196]:
price = get_price_function(price_NOK, tF, 1)
γ = 1.225*1020*3

(Ts3,qs3,Ws3,costs3) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=3.062371700997932e6, k = 1
α = 10, new_cost = 1.088772897234485e7, L2fgrad = 10391.189992044807, interm = 0.0
α = 5.0, new_cost = 6.200814419100747e6, L2fgrad = 10391.189992044807, interm = 0.0
α = 2.5, new_cost = 942393.8815304266, L2fgrad = 10391.189992044807, interm = 0.0
entered for loop, E=942393.8815304266, k = 2
α = 10, new_cost = 3.062371700997932e6, L2fgrad = 3666.6310918446893, interm = 0.0
α = 5.0, new_cost = 1.4574351566956935e6, L2fgrad = 3666.6310918446893, interm = 0.0
α = 2.5, new_cost = 665415.2891397068, L2fgrad = 3666.6310918446893, interm = 0.0
entered for loop, E=665415.2891397068, k = 3
α = 10, new_cost = 1.2480536022944455e6, L2fgrad = 1393.369746691752, interm = 0.0
α = 5.0, new_cost = 694803.4408140454, L2fgrad = 1393.369746691752, interm = 0.0
α = 2.5, new_cost = 614626.613751116, L2fgrad = 1393.369746691752, interm = 0.0
entered for loop, E=614626.613751116, k = 4
α = 10, new_cost = 618038.4968963676, L2fgrad = 713.26

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (28710.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28730.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28750.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28770.0, SingleFieldFEFunction()), (28780.0, SingleFieldFEFunction()), (28790.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [197]:
price = get_price_function(price_NOK, tF, 1)

# c1_real = (plot_costs(costs3))
c1_real = (plot_costs2(costs3))

CairoMakie.save("price_real_convergence.png", c1_real)

Q_int = get_control_int(qs3)
T_int = get_temperature_int(Ts3)
c2_real = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_real_result.png", c2_real)


CairoMakie.Screen{IMAGE}


In [188]:
price = t -> 1
γ = 1.225*1020*2

(Ts4,qs4,Ws4,costs4) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=100,
armijoparas=(ρ=0.5, α_0=20, α_min=1e-5, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=2.0415811339986213e6, k = 1
α = 20, new_cost = 7.589165507728794e6, L2fgrad = 7217.325982076949, interm = 0.0
α = 10.0, new_cost = 5.314706799875179e6, L2fgrad = 7217.325982076949, interm = 0.0
α = 5.0, new_cost = 1.4094178730076028e6, L2fgrad = 7217.325982076949, interm = 0.0
entered for loop, E=1.4094178730076028e6, k = 2
α = 20, new_cost = 2.0415811339986213e6, L2fgrad = 5767.993348318076, interm = 0.0
α = 10.0, new_cost = 2.0415811339986213e6, L2fgrad = 5767.993348318076, interm = 0.0
α = 5.0, new_cost = 999095.042261385, L2fgrad = 5767.993348318076, interm = 0.0
entered for loop, E=999095.042261385, k = 3
α = 20, new_cost = 6.315417130375541e6, L2fgrad = 4613.009668670966, interm = 0.0
α = 10.0, new_cost = 3.3877346271364335e6, L2fgrad = 4613.009668670966, interm = 0.0
α = 5.0, new_cost = 737137.803364968, L2fgrad = 4613.009668670966, interm = 0.0
entered for loop, E=737137.803364968, k = 4
α = 20, new_cost = 2.0415811339986213e6, L2fgrad 

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (28710.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28730.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28750.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28770.0, SingleFieldFEFunction()), (28780.0, SingleFieldFEFunction()), (28790.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [189]:
price = t -> 1

# c1_const = (plot_costs(costs4))
c1_const = (plot_costs2(costs4))

CairoMakie.save("price_const_convergence.png", c1_const)

Q_int = get_control_int(qs4)
T_int = get_temperature_int(Ts4)
c2_const = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_const_result.png", c2_const)


CairoMakie.Screen{IMAGE}


In [190]:
# real_price = 0
# for (t,QQ) in qs2
#     real_price+=Δt*price(t)*∑(∫(QQ)*dΩ)
# end
# temp_difference =  0
# tmp = last(Ts2)[2]-Tfin
# temp_difference = ∑(∫(tmp)*dΩ)
# println(real_price)
# println(temp_difference/(width*height))

In [191]:
# # Opprett en mappe for midlertidige filer hvis den ikke eksisterer
# if !isdir("tmp")
#     mkdir("tmp")
# end

# # Initialtilstanden og resten av tidsløsningene
# # uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# # uh  = solution[2:end]  # De resterende løsningene


# uh = qs
# # Lagre resultater i Paraview-format (VTK)
# createpvd("resultsOutisde") do pvd
#     # Første løsning (t=0)
#     # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

#     # Lagrer løsninger for hver tidssteg
#     count = 0
#     for (tn, uhn) in uh
#         if count%2 == 0
#             pvd[tn] = createvtk(Ω, "tmp/results_$count.vtu", cellfields=["u" => uhn])
#         end
#         count+=1
#     end
# end


In [192]:
# better version of L2 norm, the scalar product and the functions to get the integrals over Q and T over time

get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)

# Result plot. Now in CairoMakie instead of Pyplot because that is already loaded. I removed the legend on the plots, changed the unit [W] to (W) and I made the function have inputs (just for me, feel free to delete that part)

function result_plot(;T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    time = LinRange(t0, tF, length(T_int))

    fig = CairoMakie.Figure(size=(600,900))

    ax = CairoMakie.Axis(fig[1,1], title="Temperature profile", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, time, T_int, color=:green) #label="Temp"

    ax=CairoMakie.Axis(fig[2,1], title="Optimal Heating Schedule", xlabel="Time (hours)", ylabel="Heating Output (W)")  
    lines!(ax, time, Q_int, color=:red)

    ax=CairoMakie.Axis(fig[3,1], title="Energy price over time", xlabel="Time (hours)", ylabel="Price (NOK)")
    lines!(ax, hour_lst, price_NOK, color=:blue)

    ax=CairoMakie.Axis(fig[4,1], title="Outside Temperature over time", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, hour_lst, T_out, color=:orange)

    return fig
end

function result_plot2(; T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    # time = LinRange(t0, tF, length(T_int))
    time = LinRange(t0, tF, length(T_int)) ./ 3600  # convert from seconds to hours
    time2 = LinRange(t0, tF, length(T_int))

    
    time_price = [price(t) for t in time2]

    fig = CairoMakie.Figure(size=(600, 900), figure_padding=(0, 5, 0, 5))

    # Formatter for temperature axis (Kelvin to Celsius)
    temp_formatter = v -> ["$(round(vi - 273.15, digits=1))" for vi in v]

    # Temperature profile
    ax = CairoMakie.Axis(
        fig[1, 1],
        title = "Temperature profile",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, time, T_int, color = :green)

    # Heating output
    ax = CairoMakie.Axis(
        fig[2, 1],
        title = "Optimal Heating Schedule",
        xlabel = "Time (hours)",
        ylabel = "Heating Output (W)"
    )
    lines!(ax, time, Q_int, color = :red)

    # Energy price
    ax = CairoMakie.Axis(
        fig[3, 1],
        title = "Energy price over time",
        xlabel = "Time (hours)",
        ylabel = "Price (NOK)"
    )
    lines!(ax, time, time_price, color = :blue)

    # Outside temperature
    ax = CairoMakie.Axis(
        fig[4, 1],
        title = "Outside Temperature over time",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, hour_lst .-10, T_out, color = :orange)

    return fig
end

function plot_costs(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost"#,
        # yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end

function plot_costs2(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost",
        yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end

# In the gradient method right after the first computation of the SE solution (that I call T now instead of y) before the for loop even starts I added a checker whether T_fin can be reached.

# if ∑(∫(last(T)[2]-Tfin)dΩ) < 0
#         println("In the given time the desired final temperature cannot be reached")
#         #return nothing
#     end

# 
# # I added 2 assamblers for the SE and AE solution in order to get better errors when this "nothing-error" occurs. Instead of assambling the T and W inside the solvers I now use that method and who would have guessed: I didn't run into that error since then...

# function assamble_SE_solution(T,t0,Tini,Trialspace)
#     #[(t0,TIni),collect((t,FEFunction(Trialspace(t),copy(get_free_dof_values(TT)))) for (t,TT) in T)...]
#     Tvek=[(t0,Tini)]
#     for (t,TT) in T
#         freedofs=copy(get_free_dof_values(TT))
#         if freedofs==nothing
#             @error("freedofs found for SE are negative for time $t")
#         end
#         push!(Tvek,(t,FEFunction(Trialspace(t),freedofs)))
#     end
#     return Tvek
# end
# function assamble_AE_solution(W,tF,W_end,Trialspace)
#     #[(t0,TIni),collect((t,FEFunction(Trialspace(t),copy(get_free_dof_values(TT)))) for (t,TT) in T)...]
#     Wvek=[(tF,W_end)]
#     for (t,WW) in W
#         freedofs=copy(get_free_dof_values(WW))
#         if freedofs==nothing
#             @error("freedofs found for AE are negative for time $t")
#         end
#         push!(Wvek,(tF-t,FEFunction(Testspace(tF-t),freedofs)))
#     end
#     return reverse!(Wvek)
# end

plot_costs2 (generic function with 1 method)